In [129]:
import numpy as np
import pandas as pd
import time
from sklearn.svm import SVC,LinearSVC
from sklearn.multiclass import OneVsRestClassifier

def readData():
    readdata = pd.read_csv("/Users/academic/Desktop/TheGrandTour/data/testData.txt", sep=",", header=None);
    data = np.array(readdata);
    data = np.delete(data, 0, 0)
    data = np.swapaxes(data,0,1)


    # Need to seperate the classification dimension:
    classIndex = np.shape(data)[0] - 1
    classification = data[classIndex]
    data = np.delete(data, classIndex, axis=0)
    data = data.astype(float)

    # make list of colours for each number:

    # make list of colours for each number:
    colour = np.chararray((len(classification)),unicode = True)
    for i in range(len(classification)):
        if classification[i] == 1:
            colour[i] = 'r'
        elif classification[i] == 2:
            colour[i] = 'b'
        elif classification[i] == 3:
            colour[i] = 'g'



    for i in range(0, np.shape(data)[0]):
        data[i,:] = data[i,:] - np.min(data[i,:])
        data[i,:] = (data[i,:]/np.ndarray.max(data[i,:]))*2 - 1
    
    return data
def shapeData():
#Clifford Torus
    #Tesseract
    a = 100
    nPoints = 32*a
    data = np.zeros((4,nPoints))
    b = np.arange(0,32*a)
    b = b % a
    b = b.dot(2/a) - 1
    for i in range(8*a):
        data[0,i] = b[i]
        data[1,i] = (-1)**(i // (a) % 2)
        data[2,i] = (-1)**(i // (2*a) % 2)
        data[3,i] = (-1)**(i // (4*a) % 2)
    for i in range(8*a,16*a):
        data[1,i] = b[i]
        data[2,i] = (-1)**(i // (a) % 2)
        data[3,i] = (-1)**(i // (2*a) % 2)
        data[0,i] = (-1)**(i // (4*a) % 2)
    for i in range(16*a,24*a):
        data[2,i] = b[i]
        data[3,i] = (-1)**(i // (a) % 2)
        data[0,i] = (-1)**(i // (2*a) % 2)
        data[1,i] = (-1)**(i // (4*a) % 2)
    for i in range(24*a,32*a):
        data[3,i] = b[i]
        data[0,i] = (-1)**(i // (a) % 2)
        data[1,i] = (-1)**(i // (2*a) % 2)
        data[2,i] = (-1)**(i // (4*a) % 2)
    return data

data = shapeData()

def getAlpha(d,stepSize):
    """
    NEEDS IMPLEMENTATION
    Should produce 1xd(d-1)/2 array of position in grand tour.

    """
    p = d*(d-1)/2     
    primeList = []
    count = 1
    while len(primeList) < p:
        count += 1
        primeBool = False
        for i in range(2, count - 1):
            if count % i == 0:
                primeBool = True
        if primeBool == False:
            irrational = (np.sqrt(count)%1)
            primeList.append(irrational)
            
    primeList = np.asarray(primeList)
    primeList = primeList.dot(stepSize)
    """
    Irrational number generation using exponentials, not being used
    p = int(d*(d-1)/2)
    alpha = np.zeros(p) #alpha(t) parameters defining grand tour in G2,d

    for i in range(0,p):
        alpha[i] = (np.exp(i) % 1) * 2 * np.pi
        
    alpha = alpha.dot(0.001)
    """
    
    return primeList


def getAngles(alpha,d):
    """""
    Inputs: 
    alpha = 1xd(d-1)/2 array defining position on grand tour
    d = dimensions of data
    Outputs a dxd array of angles required for the transformation
    """
    theta = np.zeros((d,d));
    i = 0;
    k = 0;
    
    while i < d-1:
        j = i + 1;
        
        while j < d:
            theta[i][j] = alpha[k];
            j += 1;
            k += 1;
    
        i+= 1;
    
    return theta;


def RotationMatrix(i, j, d, theta):
    """
    Inputs:
    i = first indicie of rotating plane
    j = second indicie of rotating plane
    d = dimensions of data
    theta = dxd array of angle of rotation of rotating plane

    Outputs a rotating matrix to rotate plane of ixj plane by theta_ij
    """
    R = np.identity(d)
    R[i,i] = np.cos(theta)
    R[i,j] = -1*np.sin(theta)
    R[j,i] = np.sin(theta)
    R[j,j] = np.cos(theta)
    
    return R


def BetaFn(d, theta):
    """
    Inputs:
    d = dimensions of data
    theta = dxd array of angle of rotation ixj plane

    Outputs the full matrix transformation for all rotations
    """
    b = RotationMatrix(0, 1, d, theta[0,1])
    i = 0
    j = 1
    for i in range(d):
        for j in range(d):
            if j <= i:
                continue
            if i==0 and j==1:
                continue
            b = np.matmul(RotationMatrix(i, j, d, theta[i,j]),b)

   
    return b


def Transform(data,alpha):
    d = np.shape(data)[0]
    theta = getAngles(alpha,d)
    b = BetaFn(d,theta)
    a = np.matmul(b,data)
    return a
    
def SVM(a,xAxisIndex,yAxisIndex):
    
    '''xData = a[xAxisIndex]
    yData = a[yAxisIndex]
    nData = np.vstack((xData, yData)).T
    clf = OneVsRestClassifier(SVC(kernel='linear'), n_jobs=-1)
    clf.fit(nData, classification)
    score = clf.score(nData, classification)'''
    score = 0;
    return score




def singleAlphaRotation(alpha,j,data,nSteps):
    """
    Alters a single parameter in the alpha vector 
    does a 2pi rotation while calculating svm at each point
    """
    xAxisIndex = 0
    yAxisIndex = 1
    d = np.shape(data)[0] #dimensions of data
    nPoints = np.shape(data)[1] #number of data points
    A = np.zeros((nSteps,d,nPoints))
    dAlpha = alpha.dot(0)
    print(j)
    dAlpha = alpha.dot(0)
    dAlpha[j] = np.pi * 2 /(nSteps-1)
    for i in range(0,nSteps):
        alpha = alpha + dAlpha
        a = Transform(data,alpha)
        A[i] = a
        
    return A

def multiAlphaRotation(jlist,data,nSteps):
    """
    Alters a single parameter in the alpha vector 
    does a 2pi rotation while calculating svm at each point
    """
    nPoints= np.shape(data)[1]
    alpha = np.ones(np.shape(getAlpha(d,1))).dot(np.pi/4)
    tData = np.zeros((0,d,nPoints))
    
    for i in range(0,np.shape(jlist)[0]):
        j = jlist[i]
        A = singleAlphaRotation(alpha,j,data,nSteps)
        tData = np.append(tData,A,axis = 0)

    
    return tData

In [130]:
nSteps = 100
d = np.shape(data)[0]
jlist = np.arange(0,6,1)
tData = multiAlphaRotation(jlist, data,nSteps)
score = np.zeros(np.shape(tData)[0])

0
1
2
3
4
5


In [131]:
import sys
import numpy as np
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui, QtCore


if __name__ == '__main__':
    app = 0
    app = QtGui.QApplication([])
    mw = QtGui.QMainWindow()
    mw.resize(800,800)
    view = pg.GraphicsLayoutWidget()  ## GraphicsView with GraphicsLayout inserted by default
    mw.setCentralWidget(view)
    mw.show()
    mw.setWindowTitle('pyqtgraph example: ScatterPlot')

    ## create four areas to add plots
    w1 = view.addPlot()


    ## There are a few different ways we can draw scatter plots; each is optimized for different types of data:


    ## 1) All spots identical and transform-invariant (top-left plot). 
    ## In this case we can get a huge performance boost by pre-rendering the spot 
    ## image and just drawing that image repeatedly.
    nPoints = np.shape(tData)[2]
    s1 = pg.ScatterPlotItem(size=10, pen=pg.mkPen(None), brush=pg.mkBrush(0, 227, 17,120))
    pos = np.zeros((2,nPoints))
    j = 0
    pos[0,:] = tData[j,0,:]
    pos[1,:] = tData[j,1,:]
    spots = [{'pos': pos[:,i], 'data': 1} for i in range(nPoints)]
    s1.addPoints(spots)
    w1.addItem(s1)
    w1.setXRange(-1,1,padding = None)
    w1.setYRange(-1,1,padding = None)
    def update():
        global j
        j += 1
        if j > np.shape(tData)[0]-1:
            j = 0
        pos[0,:] = tData[j,0,:]
        pos[1,:] = tData[j,1,:]
        spots = [{'pos': pos[:,i], 'data': 1} for i in range(nPoints)]
        s1.setData(spots)

    t = QtCore.QTimer()
    t.timeout.connect(update)
    t.start(10)

    sys.exit(app.exec_())
        


SystemExit: 0

/Users/karanmukhi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [123]:
alpha = getAlpha(d,1)
alpha

array([0.41421356, 0.73205081, 0.23606798, 0.64575131, 0.31662479,
       0.60555128])

In [124]:
np.shape(alpha)

(6,)

In [125]:
np.ones(6)

array([1., 1., 1., 1., 1., 1.])